<a href="https://colab.research.google.com/github/thenukan/Covid_19-detection---using-Xray/blob/main/Covid_19_Xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#data set: http://cb.lk/covid_19

!wget http://cb.lk/covid_19

In [ ]:
! unzip covid_19

In [4]:
train_path=   "/content/CovidDataset/Train"
val_patha=    "/content/CovidDataset/Val"

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import  image

In [6]:
# CNN Based Model in keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))


model.compile(loss=keras.losses.binary_crossentropy,optimizer ="adam", metrics=["accuracy"])


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                    

In [8]:
# Train from scratch
train_datagen=image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,zoom_range=0.2,
    horizontal_flip=True,

)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
    "CovidDataset/Train",
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'

)

Found 224 images belonging to 2 classes.


In [10]:
validation_generator = test_dataset.flow_from_directory(
    "CovidDataset/Val",
     target_size=(224,224),
     batch_size=32,
    class_mode='binary'
)

Found 60 images belonging to 2 classes.


In [11]:
from datetime import datetime 
start = datetime.now()
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=7,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=2

)
duration = datetime.now() - start
print("Training completed in time: ", duration)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
7/7 [==============================] - 24s 2s/step - loss: 0.8843 - accuracy: 0.5491 - val_loss: 0.6867 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 11s 2s/step - loss: 0.5978 - accuracy: 0.7143 - val_loss: 0.5446 - val_accuracy: 0.9167
Epoch 3/10
7/7 [==============================] - 11s 2s/step - loss: 0.4391 - accuracy: 0.7946 - val_loss: 0.3093 - val_accuracy: 0.9500
Epoch 4/10
7/7 [==============================] - 11s 2s/step - loss: 0.3281 - accuracy: 0.8973 - val_loss: 0.1868 - val_accuracy: 0.9500
Epoch 5/10
7/7 [==============================] - 12s 2s/step - loss: 0.3576 - accuracy: 0.8661 - val_loss: 0.3560 - val_accuracy: 0.9500
Epoch 6/10
7/7 [==============================] - 11s 2s/step - loss: 0.2796 - accuracy: 0.9196 - val_loss: 0.1806 - val_accuracy: 0.9667
Epoch 7/10
7/7 [==============================] - 11s 2s/step - loss: 0.1640 - accuracy: 0.9375 - val_loss: 0.0866 - val_accuracy: 0.9833
Epoch 8/10
7/7 [==================

In [12]:
model.save("/content/drive/MyDrive/Covid/model-adv.h5")

In [13]:
model.evaluate_generator(train_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[0.13966940343379974, 0.96875]

In [14]:
model.evaluate_generator(validation_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[0.11595975607633591, 0.9833333492279053]

Test images
```

```



In [15]:
model = load_model("/content/drive/MyDrive/Covid/model-adv.h5")

In [16]:
import os

In [17]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [18]:
y_actual=[]
y_test=[]